# 1.0 Um problema de classificação de ponta-a-ponta usando NLP

## 1.1 Descrição do dataset

O dataset contém informações gerais de 5.000 processos julgados
nos Juizados Especiais Federais dentro das Seções Judiciárias do Tribunal Regional Federal da 5a Região (https://www.trf5.jus.br). Os dados são oriundos da raspagem da consulta pública processual. Além disso, ele possui 46 colunas, das quais duas possuem texto livre:
"conteudo_sentenca" e "conteudo_acordao".

O dataset pode ser baixado no link a seguir: https://jacob.al/dataset_juizados 

Ao longo dos notebooks, vão ser realizados os seguintes passos:

1. Importação do dataset **(concluído)**
2. Análise exploratória dos dados **(concluído)**
3. Pré-processamento **(concluído)**
4. Verificação dos dados **(concluído)**
5. Segregação dos dados
6. Treinamento
7. Teste

<center><img width="600" src="https://drive.google.com/uc?export=view&id=1fKGuR5U5ECf7On6Zo1UWzAIWZrMmZnGc"></center>


## 1.2 Instalação e importação das bibliotecas e configuração do wandb

In [ ]:
!pip install wandb

In [ ]:
import logging
import tempfile
import pandas as pd
import os
import wandb
from sklearn.model_selection import train_test_split

In [ ]:
# Login to Weights & Biases
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## 1.3 Data Segregation

In [ ]:
# variáveis globais

# proporção utilizada para dividir entre dados de teste e de treinamento
test_size = 0.30

# Semente utilizada para propósitos de reproducibilidade
seed = 41

# coluna de referência para dividir os dados
stratify = "assunto_cnj"

# nome do artefato de entrada
artifact_input_name = "nlp_bolsa/preprocessed_data.csv:latest"

# tipo do artefato
artifact_type = "segregated_data"

In [ ]:
# configuração do logging
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(message)s",
                    datefmt='%d-%m-%Y %H:%M:%S')

# objeto do logging
logger = logging.getLogger()

# Inicialização do projeto do wandb
run = wandb.init(project="nlp_bolsa", job_type="split_data")

logger.info("Downloading and reading artifact")
artifact = run.use_artifact(artifact_input_name)
artifact_path = artifact.file()
df = pd.read_csv(artifact_path)

# Divisão entre treinamento e teste
logger.info("Splitting data into train and test")
splits = {}

splits["train"], splits["test"] = train_test_split(df,
                                                   test_size=test_size,
                                                   random_state=seed,
                                                   stratify=df[stratify])

# Salva os artefatos em um diretório temporário
with tempfile.TemporaryDirectory() as tmp_dir:

    for split, df in splits.items():

        # Cria o nome do artefato a partir do nome da divisão
        # 
        artifact_name = f"{split}.csv"

        # Pega o caminho da pasta temporária no disco
        temp_path = os.path.join(tmp_dir, artifact_name)

        logger.info(f"Uploading the {split} dataset to {artifact_name}")

        # Salva o arquivo e faz o upload no Wandb
        df.to_csv(temp_path,index=False)

        artifact = wandb.Artifact(name=artifact_name,
                                  type=artifact_type,
                                  description=f"{split} split of dataset {artifact_input_name}",
        )
        artifact.add_file(temp_path)

        logger.info("Logging artifact")
        run.log_artifact(artifact)

        # Espera até o o fim do upload do artefato
        artifact.wait()

17-07-2022 12:05:17 Downloading and reading artifact
17-07-2022 12:05:19 Splitting data into train and test
17-07-2022 12:05:19 Uploading the train dataset to train.csv
17-07-2022 12:05:19 Logging artifact
17-07-2022 12:05:21 Uploading the test dataset to test.csv
17-07-2022 12:05:21 Logging artifact
